In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_score, cross_val_predict
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
# data

# import data
df = pd.read_csv('../../#task1-datacollection/OUTPUT_WBI_exposer_cyclones_v3.csv')
df.columns = [e.lower().replace(' ','_') for e in df.columns]

# separate X and y
X = df.drop(['total_affected'], axis=1)
y = df['total_affected']

# types of features
num_features = list(df.loc[:, df.dtypes != object].columns)
cat_features = list(df.loc[:, df.dtypes == object].columns)

In [3]:
# custom transformers

class DataFrameSelector(BaseEstimator, TransformerMixin):
    # selects a slice of the dataset based on feature_names
    def __init__(self, feature_names):
        self.feature_names = feature_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[set(X.columns).intersection(self.feature_names)]


class TransformerCat(BaseEstimator, TransformerMixin):
    # transforms categorical features
    
    def __init__(self):
        self.columns_to_drop = []

    def fit(self, X, y=None):
        self.columns_to_drop= ['sid', 'name', 'iso', 'sub_basin', 'nature', 'iso_time',
       'coords', 'in_susan', 'year_tiff_c']
        return self
    
    def _get_income_level(self, income):
        order = {'High': 4, 'High_Middle': 3, 'Low_Middle': 2, 'Low': 1 }
        return order[income]
        
    def transform(self, X, y=None):
        X['income_level_final'] = X['income_level_final'].apply(self._get_income_level)
        X = X.drop(self.columns_to_drop, axis = 1)
        print(X.columns)
        return X
    
class TransformerNum(BaseEstimator, TransformerMixin):
    # transforms numerical features
    
    def __init__(self):
        self.columns_to_drop = []

    def fit(self, X, y=None):
        self.columns_to_drop = ['unnamed:_0', 'year', 'total_hrs', 'day_hrs', 'night_hrs',
                                '34kn_pop', '34kn_assets', '64kn_pop', '64kn_assets','total_damage_(000$)', 'total_deaths',
                               'air_transport,_freight_(million_ton-km)',
                               'arable_land_(hectares_per_person)', 'cereal_yield_(kg_per_hectare)',
                               'food_production_index_(2004-2006_=_100)', 'gdp_growth_(annual_%)','net_flows_from_un_agencies_us$',
                               'mobile_cellular_subscriptions_(per_100_people)',
                               'adjusted_savings:_education_expenditure_(%_of_gni)',
                               'population_2000', 'population_2005',
                               'population_2010', 'population_2015', 'population_2020', 'in_wbi',
                               'pop_max_34', 'pop_max_50', 'pop_max_64', 'numeric', 'pop_max_34_adj',
                               'pop_max_50_adj', 'year_tiff', 'coef_year']
        return self
        
    def transform(self, X, y=None):
        X = X.drop(self.columns_to_drop, axis = 1)
        return X


In [4]:
# preprocess pipeline

preprocess_pipeline = FeatureUnion(transformer_list =[
        ("num_pipeline", Pipeline([
            ('selector', DataFrameSelector(num_features)),
            ('num_transformer', TransformerNum()),
            ('imputer', SimpleImputer(strategy="most_frequent")),
            ("scaler", StandardScaler())
             ])),
        ("cat_pipeline", Pipeline([
            ('selector', DataFrameSelector(cat_features)),
            ('cat_transformer', TransformerCat()),
            ('imputer', SimpleImputer(strategy="most_frequent")),
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse = False)), # TODO: not encode income_level_final
            ("scaler", StandardScaler())
            ]))])

In [5]:
class ModelEvaluator:
    # evaluates different models using cross validation on the whole dataset
    
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def evaluate_models(self, models):
        for model_name, model in models.items():
            print(f'***{model_name}***')
            self._evaluate_model(model_name, model)
            print('\n')
            
    def _display_scores(self, scores):
            print("Scores:", scores)
            print("Mean:", scores.mean())
            print("Standard deviation:", scores.std())

    def _evaluate_model(self, model_name, model):
        transformed_X = preprocess_pipeline.fit_transform(self.X)
        scores = cross_val_score(model, transformed_X, self.y, scoring="neg_mean_squared_error", cv=10)
        tree_rmse_scores = np.sqrt(-scores)
        self._display_scores(tree_rmse_scores)     

In [6]:
# grid search random_forest

param_grid = [
    {'n_estimators': list(range(10,200,30)), 'max_features': list(range(2, 10, 2)), 'min_samples_leaf':[2,4,6]},
    {'bootstrap': [False], 'n_estimators': list(range(10,200,30)), 'max_features': list(range(2, 10, 2)), 'min_samples_leaf':[2,4,6]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
grid_search_forest = GridSearchCV(forest_reg, param_grid, cv=6, scoring='neg_mean_squared_error', return_train_score=True, iid = True, verbose=20)
transformed_X = preprocess_pipeline.fit_transform(X)
grid_search_forest.fit(transformed_X, y)

print(grid_search_forest.score(transformed_X, y))
print(grid_search_forest.best_params_)

Index(['basin', 'income_level_final'], dtype='object')
Fitting 6 folds for each of 168 candidates, totalling 1008 fits
[CV] max_features=2, min_samples_leaf=2, n_estimators=10 .............
[CV]  max_features=2, min_samples_leaf=2, n_estimators=10, score=(train=-1952890264403.983, test=-1122379331860.334), total=   0.0s
[CV] max_features=2, min_samples_leaf=2, n_estimators=10 .............
[CV]  max_features=2, min_samples_leaf=2, n_estimators=10, score=(train=-1944632713548.494, test=-1832726104284.906), total=   0.0s
[CV] max_features=2, min_samples_leaf=2, n_estimators=10 .............
[CV]  max_features=2, min_samples_leaf=2, n_estimators=10, score=(train=-1879659145978.595, test=-2457356143563.456), total=   0.0s
[CV] max_features=2, min_samples_leaf=2, n_estimators=10 .............
[CV]  max_features=2, min_samples_leaf=2, n_estimators=10, score=(train=-1086248730179.304, test=-8229227963929.140), total=   0.0s
[CV] max_features=2, min_samples_leaf=2, n_estimators=10 ............

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.1s remaining:    0.0s


[CV]  max_features=2, min_samples_leaf=2, n_estimators=40, score=(train=-1767884808753.595, test=-1681952989201.200), total=   0.0s
[CV] max_features=2, min_samples_leaf=2, n_estimators=40 .............
[CV]  max_features=2, min_samples_leaf=2, n_estimators=40, score=(train=-1688731278558.806, test=-2441055404734.316), total=   0.1s
[CV] max_features=2, min_samples_leaf=2, n_estimators=40 .............
[CV]  max_features=2, min_samples_leaf=2, n_estimators=40, score=(train=-1128427219812.265, test=-8600899419567.726), total=   0.0s
[CV] max_features=2, min_samples_leaf=2, n_estimators=40 .............
[CV]  max_features=2, min_samples_leaf=2, n_estimators=40, score=(train=-1589545226856.272, test=-3908364775429.756), total=   0.0s
[CV] max_features=2, min_samples_leaf=2, n_estimators=40 .............


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.4s remaining:    0.0s


[CV]  max_features=2, min_samples_leaf=2, n_estimators=40, score=(train=-1483251276207.388, test=-3665064595595.898), total=   0.0s
[CV] max_features=2, min_samples_leaf=2, n_estimators=70 .............
[CV]  max_features=2, min_samples_leaf=2, n_estimators=70, score=(train=-1730355015845.103, test=-1076488848427.935), total=   0.1s
[CV] max_features=2, min_samples_leaf=2, n_estimators=70 .............
[CV]  max_features=2, min_samples_leaf=2, n_estimators=70, score=(train=-1665863728724.799, test=-1622484581601.118), total=   0.1s
[CV] max_features=2, min_samples_leaf=2, n_estimators=70 .............


...

[CV]  bootstrap=False, max_features=6, min_samples_leaf=6, n_estimators=40, score=(train=-1981820391872.808, test=-998705679800.179), total=   0.1s
[CV] bootstrap=False, max_features=6, min_samples_leaf=6, n_estimators=40 
[CV]  bootstrap=False, max_features=6, min_samples_leaf=6, n_estimators=40, score=(train=-1912577848534.535, test=-1834623603337.135), total=   0.1s
[CV] bootstrap=False, max_features=6, min_samples_leaf=6, n_estimators=40 
[CV]  bootstrap=False, max_features=6, min_samples_leaf=6, n_estimators=40, score=(train=-1883189797725.898, test=-2663898139700.805), total=   0.1s
[CV] bootstrap=False, max_features=6, min_samples_leaf=6, n_estimators=40 
[CV]  bootstrap=False, max_features=6, min_samples_leaf=6, n_estimators=40, score=(train=-1294490603203.470, test=-8550753072588.786), total=   0.1s
[CV] bootstrap=False, max_features=6, min_samples_leaf=6, n_estimators=40 
[CV]  bootstrap=False, max_features=6, min_samples_leaf=6, n_estimators=40, score=(train=-1778192485594.9

[CV]  bootstrap=False, max_features=8, min_samples_leaf=2, n_estimators=10, score=(train=-478652247562.176, test=-3988692618537.810), total=   0.0s
[CV] bootstrap=False, max_features=8, min_samples_leaf=2, n_estimators=40 
[CV]  bootstrap=False, max_features=8, min_samples_leaf=2, n_estimators=40, score=(train=-546557866521.167, test=-1125914845910.736), total=   0.1s
[CV] bootstrap=False, max_features=8, min_samples_leaf=2, n_estimators=40 
[CV]  bootstrap=False, max_features=8, min_samples_leaf=2, n_estimators=40, score=(train=-469523163475.596, test=-2944658904806.251), total=   0.1s
[CV] bootstrap=False, max_features=8, min_samples_leaf=2, n_estimators=40 
[CV]  bootstrap=False, max_features=8, min_samples_leaf=2, n_estimators=40, score=(train=-518800684175.856, test=-3048668261622.874), total=   0.1s
[CV] bootstrap=False, max_features=8, min_samples_leaf=2, n_estimators=40 
[CV]  bootstrap=False, max_features=8, min_samples_leaf=2, n_estimators=40, score=(train=-328660786418.026, 

[CV]  bootstrap=False, max_features=8, min_samples_leaf=4, n_estimators=40, score=(train=-1432822905039.776, test=-1058387126735.655), total=   0.1s
[CV] bootstrap=False, max_features=8, min_samples_leaf=4, n_estimators=40 
[CV]  bootstrap=False, max_features=8, min_samples_leaf=4, n_estimators=40, score=(train=-1411760244760.959, test=-2242332955850.671), total=   0.1s
[CV] bootstrap=False, max_features=8, min_samples_leaf=4, n_estimators=40 
[CV]  bootstrap=False, max_features=8, min_samples_leaf=4, n_estimators=40, score=(train=-1395601525372.501, test=-3068639049412.573), total=   0.1s
[CV] bootstrap=False, max_features=8, min_samples_leaf=4, n_estimators=40 
[CV]  bootstrap=False, max_features=8, min_samples_leaf=4, n_estimators=40, score=(train=-891476812749.976, test=-8406139546308.864), total=   0.1s
[CV] bootstrap=False, max_features=8, min_samples_leaf=4, n_estimators=40 
[CV]  bootstrap=False, max_features=8, min_samples_leaf=4, n_estimators=40, score=(train=-1320157924129.5

[CV]  bootstrap=False, max_features=8, min_samples_leaf=6, n_estimators=40, score=(train=-1928873966108.976, test=-1023032543533.223), total=   0.1s
[CV] bootstrap=False, max_features=8, min_samples_leaf=6, n_estimators=40 
[CV]  bootstrap=False, max_features=8, min_samples_leaf=6, n_estimators=40, score=(train=-1830912659236.188, test=-2206688980682.568), total=   0.1s
[CV] bootstrap=False, max_features=8, min_samples_leaf=6, n_estimators=40 
[CV]  bootstrap=False, max_features=8, min_samples_leaf=6, n_estimators=40, score=(train=-1882925873192.715, test=-2877501532093.169), total=   0.1s
[CV] bootstrap=False, max_features=8, min_samples_leaf=6, n_estimators=40 
[CV]  bootstrap=False, max_features=8, min_samples_leaf=6, n_estimators=40, score=(train=-1260995829338.335, test=-8841054320640.588), total=   0.1s
[CV] bootstrap=False, max_features=8, min_samples_leaf=6, n_estimators=40 
[CV]  bootstrap=False, max_features=8, min_samples_leaf=6, n_estimators=40, score=(train=-1741530674374.

[Parallel(n_jobs=1)]: Done 1008 out of 1008 | elapsed:  3.3min finished


In [7]:
# evaluate models 

models = {'forest_grid_search': grid_search_forest.best_estimator_, 
          'forest_1': RandomForestRegressor(n_estimators=3, random_state=42), 
          'forest_2':RandomForestRegressor(max_depth=5, n_estimators=10, max_features=9, random_state=42)}

random_forest_evaluator = ModelEvaluator(X, y)

random_forest_evaluator.evaluate_models(models)

***forest_grid_search***
Index(['basin', 'income_level_final'], dtype='object')
Scores: [ 803822.23350209 1166260.83138995 1371370.6594198  1607944.87480684
 1170160.25182717 3770440.35286206 1504366.02073208 1092923.55546392
 2219815.69575327 2188448.69041575]
Mean: 1689555.3166172914
Standard deviation: 817707.2166923254


***forest_1***
Index(['basin', 'income_level_final'], dtype='object')
Scores: [1285644.09901694 1191174.46678079 2290930.60438599 2079677.4321966
 2179793.19728161 4057866.37765171 1328287.50550625 1351688.90955074
 2199649.19200848 3156808.39235801]
Mean: 2112152.017673712
Standard deviation: 874438.1898887912


***forest_2***
Index(['basin', 'income_level_final'], dtype='object')
Scores: [1251088.96066901 1226099.07708944 1522147.21741576 1751063.81345957
 1241616.0936574  3804063.29749966 1652006.29579458 1457939.5109553
 2309262.59423097 2335759.9899946 ]
Mean: 1855104.6850766286
Standard deviation: 754536.5606148882


